In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
# libraries
import pandas as pd
from itertools import permutations

from word_to_biocode import WordToBiocode

import warnings 
warnings.simplefilter('ignore')

- Русский алфавит - переводим в двоичную систему (6 цифр)
- Переводим в четверичную систему (3 цифры)
- Соответствующие нуклеотиды - тоже три цифры
- Тогда наша закодированная фамилия будет длиной 3 * [длина нашей фамилии]

При равномерном распределении нуклеотидов вероятность встретить именно нашу фамилию:
$$\frac{1}{4^{\text{length of 4-base surname}}} = \frac{1}{4^{3 \cdot \text{length of surname}}}$$

Наш геном размера $3.2 \cdot 10^9$ нуклеотидов. В геноме длиной 3.2 млрд нуклеотидов можно разместить такую последовательность начиная с любой из позиций от 1 до $3.2 \cdot 10^9$:
$$3.2 \cdot 10^9 - 6 + 1 \approx 3.2 \cdot 10^9$$

Чтобы найти,** сколько раз можно ожидать появления нашей закодированной последовательности**, умножаем количество возможных позиций на вероятность встретить последовательность в одной позиции:
$$\frac{3.2 \cdot 10^9}{4^{3 \cdot \text{length of surname}}}$$

Вероятность встретить хотя бы одну такую последовательность:
$$P = 1 - (1 - \frac{1}{4^{3 \cdot \text{length of surname}}})^{3.2 \cdot 10^9}$$

Так как у нас получившаяся последовательность (закодированное слово) кратно трем - мы еще можем преобразовать ее в белок!

In [38]:
sequences = []
protein_sequences = []
convertation = []

word = 'Ракитько'

for nucl in list(permutations(['A', 'T', 'G', 'C'])):
    wordfinder = WordToBiocode(lang='rus', nucl=nucl)

    seq_word = wordfinder.convert_to_nucleotide(word)
    protein_word = wordfinder.convert_to_protein(word)

    sequences.append(seq_word)
    protein_sequences.append(protein_word)
    convertation.append(''.join(nucl))

    print(wordfinder.nucl_to_fourth)
    print(f'DNA: {seq_word}\nPROTEIN: {protein_word}')
    print('='*60)

{'A': '0', 'T': '1', 'G': '2', 'C': '3'}
DNA: TATAAAAGCAGTTACTCTAGCACC
PROTEIN: YKSSYSST
{'A': '0', 'T': '1', 'C': '2', 'G': '3'}
DNA: TATAAAACGACTTAGTGTACGAGG
PROTEIN: YKTT*CTR
{'A': '0', 'G': '1', 'T': '2', 'C': '3'}
DNA: GAGAAAATCATGGACGCGATCACC
PROTEIN: EKIMDAIT
{'A': '0', 'G': '1', 'C': '2', 'T': '3'}
DNA: GAGAAAACTACGGATGTGACTATT
PROTEIN: EKTTDVTI
{'A': '0', 'C': '1', 'T': '2', 'G': '3'}
DNA: CACAAAATGATCCAGCGCATGAGG
PROTEIN: HKMIQRMR
{'A': '0', 'C': '1', 'G': '2', 'T': '3'}
DNA: CACAAAAGTAGCCATCTCAGTATT
PROTEIN: HKSSHLSI
{'T': '0', 'A': '1', 'G': '2', 'C': '3'}
DNA: ATATTTTGCTGAATCACATGCTCC
PROTEIN: IFC*ITCS
{'T': '0', 'A': '1', 'C': '2', 'G': '3'}
DNA: ATATTTTCGTCAATGAGATCGTGG
PROTEIN: IFSSMRSW
{'T': '0', 'G': '1', 'A': '2', 'C': '3'}
DNA: GTGTTTTACTAGGTCGCGTACTCC
PROTEIN: VFY*VAYS
{'T': '0', 'G': '1', 'C': '2', 'A': '3'}
DNA: GTGTTTTCATCGGTAGAGTCATAA
PROTEIN: VFSSVES*
{'T': '0', 'C': '1', 'A': '2', 'G': '3'}
DNA: CTCTTTTAGTACCTGCGCTAGTGG
PROTEIN: LF*YLR*W
{'T': '0', 'C': '1', 

In [39]:
data = pd.DataFrame({"encoding": convertation, 
              "DNA_seq": sequences, 
              "PROTEIN_seq": protein_sequences})

In [40]:
data

,encoding,DNA_seq,PROTEIN_seq
0,ATGC,TATAAAAGCAGTTACTCTAGCACC,YKSSYSST
1,ATCG,TATAAAACGACTTAGTGTACGAGG,YKTT*CTR
2,AGTC,GAGAAAATCATGGACGCGATCACC,EKIMDAIT
3,AGCT,GAGAAAACTACGGATGTGACTATT,EKTTDVTI
4,ACTG,CACAAAATGATCCAGCGCATGAGG,HKMIQRMR
5,ACGT,CACAAAAGTAGCCATCTCAGTATT,HKSSHLSI
6,TAGC,ATATTTTGCTGAATCACATGCTCC,IFC*ITCS
7,TACG,ATATTTTCGTCAATGAGATCGTGG,IFSSMRSW
8,TGAC,GTGTTTTACTAGGTCGCGTACTCC,VFY*VAYS
9,TGCA,GTGTTTTCATCGGTAGAGTCATAA,VFSSVES*
